# Data Science Capstone Project
## Peer-graded Assigment: Segmenting and Clustering Neighborhoods in Toronto


## Overall Process - Part 2

The creation of the Dataframe with Toronto neighborhoods will proceed along the following steps:

1. Initialize Libraries
2. Read the Wiki page "List of Postal Codes of Canada: M"
3. Create a temporary dataframe with all neighborhoods with assigned postal code
4. Create a secomnd temprary dataframe with neighborhoods aggregated by Postcode.
5. Create final dataframe of Toronto Neoghborhoods

Notes: Steps 1 to 5 are similar to the same steps in part1;
    But Steps 5 was enhanced to insert the Latitude and Longitude columns

## 1. Initialize Libraries

In [1]:
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
#!pip install geocoder
#import geocoder

#Note:
# geocoder.google(address) did not return latitude/Longitude coordinates
# geopy was used instead (see 2 lines below)

In [4]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

## 2. Read the Wiki page "List of Postal Codes of Canada: M"

In [5]:
page_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
page_html = BeautifulSoup(page_text,"html.parser")

In [6]:
page_table = page_html.find('tbody')
table_rows = page_table.findAll('tr')

## 3. Create a temporary dataframe with all neighborhoods with assigned postal code

In [7]:
neighborhood_names = ['Postcode', 'Borough', 'Neighborhood']

In [8]:
neighborhoods_temp = pd.DataFrame(columns=neighborhood_names)

for i in range(1,len(table_rows)):
    table_col = table_rows[i].text.split('\n')
    if table_col[2] == 'Not assigned':
        continue
        
    neighborhoods_temp = neighborhoods_temp.append({ \
        'Postcode': table_col[1],
        'Borough': table_col[2],
        'Neighborhood': table_col[3]}, ignore_index=True)

In [9]:
neighborhoods_temp.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## 4. Create a second temporary dataframe with neighborhoods aggregated by Postcode and Borough.

In [10]:
df = neighborhoods_temp.groupby( [ "Postcode", "Borough"] ).agg(lambda x : ', '.join(x))

## 5. Create final dataframe of Toronto Neighborhoods

In [11]:
neighborhood_coords = ['Postcode', 'Borough', 'Neighborhood','Latitude','Longitude']
neighborhoods = pd.DataFrame(columns=neighborhood_coords)

In [12]:
neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude


In [13]:
# For each row in the df Dataframe,
# append the row to neighborhood Dataframe

for ix in range(0,df.size):
    neighborhoods = neighborhoods.append({ \
       'Postcode': df.index[ix][0],
       'Borough': df.index[ix][1],
       'Neighborhood': df.iloc[ix,0]}, ignore_index=True)

In [17]:
# Inject Latitude and Longitude for each neighborhood

import time
geolocator = Nominatim()

for ix in range(0,neighborhoods.size -1):
    
    postcode = neighborhoods.Postcode[ix]
    location = geolocator.geocode('{}, Toronto, Ontario'.format(postcode))

    if location is not None:
        neighborhoods.loc[ix, ['Latitude']] = location.latitude
        neighborhoods.loc[ix, ['Longitude']] = location.longitude

    time.sleep(1)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


## 6. Finally show Toronto Neighborhood list as per requirements, sorted by Postcode

In [19]:
neighborhoods.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.654,-79.3872
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.654,-79.3872
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,43.7657,-79.2219
4,M1H,Scarborough,Cedarbrae,NaN,NaN


In [21]:
neighborhoods.shape

(103, 5)

In [23]:
# Note: The module
#       geopy.geocoders import Nominatim 
# to convert an address into latitude and longitude values is very unreliable.
# It time out without returning real data.
# As a result, many neighborhoods appears with latitude/longitude cells with NaN values.

neighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.654,-79.3872
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.654,-79.3872
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",NaN,NaN
3,M1G,Scarborough,Woburn,43.7657,-79.2219
4,M1H,Scarborough,Cedarbrae,NaN,NaN
5,M1J,Scarborough,Scarborough Village,NaN,NaN
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",NaN,NaN
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",NaN,NaN
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",NaN,NaN
9,M1N,Scarborough,"Birch Cliff, Cliffside West",NaN,NaN
